# Burseras and Friends
## How´s the environment ?
## Which are their friends ? 

In this example we will explore how to query and select a Node (Bursera, Node Type : Family) in the Tree Of Life.
We will extract the associated Cells and explore which kind of other taxa share that space. 
Let´s start!

![Burcera Environment](http://www.geographylists.com/mex_pv_forest.jpg "Bursera and its environment")


### Import modules libraries and load ploting style

In [1]:
%matplotlib inline
import sys
sys.path.append('/apps')
import django
django.setup()
from drivers.tree_builder import TreeNeo
from drivers.graph_models import TreeNode, Order, Family, graph

### Select the Node Bursera

Bursera is a Family of Plants. We want to select a Node in the Tree Of Life that is of the type Family and the name starts 
with "Burser". The expression "_.name=~STRING" means that we will look into the attribute "name" using a regular expression 
"=~" followed by any number of characters ".*"

In [2]:
bursera = list(Family.select(graph).where("_.name=~'Burser.*'")).pop()

The result is a list. Given that there is only one Node of the type Family that has the name "Burser" we can extract
the only element of the list using the standard method pop().

In [3]:
bursera

<TreeNode type: Family id = 343770 name: Burseraceae>

### Inspect taxonomy

In [4]:
bursera

<TreeNode type: Family id = 343770 name: Burseraceae>

In [5]:
bursera.getParent()

<TreeNode type: Order id = 933 name: Sapindales>

In [6]:
magno = bursera.getParent().getParent()

In [7]:
ll = list(magno.parent_link)

In [8]:
list(magno.parent_link)

[<TreeNode type: Phylum id = 49 name: Magnoliophyta>]

The Tree Node is an object and has defined methods (refer to the documentation). One of this methods extract the cells in the spatial lattice that has associated Bursera occurrences. 

In [9]:
cells_with_burseras = bursera.cells

The result is a generator object. This is helpfull when the number of cells is big and surpasses the capacity of the machine
or the analyst.

In [10]:
cells_with_burseras

<generator object __iter__ at 0x7fe6d6508c30>

In this example we want to get only the first 10 cells.

In [11]:
cells_with_burseras = list(cells_with_burseras)

In [12]:
len(cells_with_burseras)

811

## *Oke Oke*!, 
Now we want to get the subtrees of Life constrainted by the geographical attribute of this cells.
For doing this, simply get the occurrences within each cell with the method *occurrencesHere()* and use this output as the constructor argument.

In [13]:
c = cells_with_burseras[0]

In [14]:
c.occurrencesHere()

[<Occurrence pk=1266943>,
 <Occurrence pk=1266931>,
 <Occurrence pk=1266975>,
 <Occurrence pk=1266893>,
 <Occurrence pk=1266913>,
 <Occurrence pk=1266863>,
 <Occurrence pk=1266877>,
 <Occurrence pk=1343013>,
 <Occurrence pk=2090835>,
 <Occurrence pk=2096496>,
 <Occurrence pk=1266886>,
 <Occurrence pk=1239196>,
 <Occurrence pk=1266953>,
 <Occurrence pk=1266903>,
 <Occurrence pk=1239323>,
 <Occurrence pk=1239209>,
 <Occurrence pk=1239207>,
 <Occurrence pk=1239264>,
 <Occurrence pk=1343005>,
 <Occurrence pk=1266841>,
 <Occurrence pk=1314821>,
 <Occurrence pk=1266851>,
 <Occurrence pk=1342675>,
 <Occurrence pk=286138>,
 <Occurrence pk=1326412>,
 <Occurrence pk=1315197>,
 <Occurrence pk=1335226>,
 <Occurrence pk=1335240>,
 <Occurrence pk=283967>,
 <Occurrence pk=1328216>,
 <Occurrence pk=1307801>,
 <Occurrence pk=284084>,
 <Occurrence pk=284152>,
 <Occurrence pk=284098>,
 <Occurrence pk=283938>,
 <Occurrence pk=284179>,
 <Occurrence pk=284028>,
 <Occurrence pk=284117>,
 <Occurrence pk=28413

## Random subselection of cells
The number of cells is very big. An option for processing is to select a random uniform set of indexes. We can achieve this with the following strategy.

1. make a random sample selection


### Let's create a random subset of 100 cells.

In [15]:
import numpy.random as rnd

In [16]:
n = 100

In [17]:
indices = rnd.randint(0,len(cells_with_burseras),n)

In [18]:
selected_cells = [ cells_with_burseras[i] for i in indices ]

### Get the subtrees within the selected cells

Let's create first a Constructor function

In [19]:
ToTree = lambda cell : TreeNeo(cell.occurrencesHere())

and map it to the selected cells

In [20]:
%time trees = map(ToTree,selected_cells)

CPU times: user 3min 7s, sys: 4.72 s, total: 3min 12s
Wall time: 4min 11s


#### Uff! it took 3 minutes, but it's a lot of data

In [50]:
trees

[<LocalTree Of Life | Root: LUCA - n.count : 484- >,
 <LocalTree Of Life | Root: LUCA - n.count : 12- >,
 <LocalTree Of Life | Root: LUCA - n.count : 571- >,
 <LocalTree Of Life | Root: LUCA - n.count : 7- >,
 <LocalTree Of Life | Root: LUCA - n.count : 35- >,
 <LocalTree Of Life | Root: LUCA - n.count : 23- >,
 <LocalTree Of Life | Root: LUCA - n.count : 46- >,
 <LocalTree Of Life | Root: LUCA - n.count : 5- >,
 <LocalTree Of Life | Root: LUCA - n.count : 30- >,
 <LocalTree Of Life | Root: LUCA - n.count : 13- >,
 <LocalTree Of Life | Root: LUCA - n.count : 250- >,
 <LocalTree Of Life | Root: LUCA - n.count : 2- >,
 <LocalTree Of Life | Root: LUCA - n.count : 31- >,
 <LocalTree Of Life | Root: LUCA - n.count : 133- >,
 <LocalTree Of Life | Root: LUCA - n.count : 1- >,
 <LocalTree Of Life | Root: LUCA - n.count : 2632- >,
 <LocalTree Of Life | Root: LUCA - n.count : 1716- >,
 <LocalTree Of Life | Root: LUCA - n.count : 15- >,
 <LocalTree Of Life | Root: LUCA - n.count : 27- >,
 <LocalT

#### Get the UNION of the these 100 trees~

In [21]:
%time big_tree = reduce(lambda a,b : a + b , trees)

INFO Merging Trees
INFO Merging Trees
INFO Merging Trees
INFO Merging Trees
INFO Merging Trees
INFO Merging Trees
INFO Merging Trees
INFO Merging Trees
INFO Merging Trees
INFO Merging Trees
INFO Merging Trees
INFO Merging Trees
INFO Merging Trees
INFO Merging Trees
INFO Merging Trees
INFO Merging Trees
INFO Merging Trees
INFO Merging Trees
INFO Merging Trees
INFO Merging Trees
INFO Merging Trees
INFO Merging Trees
INFO Merging Trees
INFO Merging Trees
INFO Merging Trees
INFO Merging Trees
INFO Merging Trees
INFO Merging Trees
INFO Merging Trees
INFO Merging Trees
INFO Merging Trees
INFO Merging Trees
INFO Merging Trees
INFO Merging Trees
INFO Merging Trees
INFO Merging Trees
INFO Merging Trees
INFO Merging Trees
INFO Merging Trees
INFO Merging Trees
INFO Merging Trees
INFO Merging Trees
INFO Merging Trees
INFO Merging Trees
INFO Merging Trees
INFO Merging Trees
INFO Merging Trees
INFO Merging Trees
INFO Merging Trees
INFO Merging Trees
INFO Merging Trees
INFO Merging Trees
INFO Merging

CPU times: user 1min 20s, sys: 492 ms, total: 1min 21s
Wall time: 1min 22s


In [22]:
big_tree.richness

22861

I told you, loads of data.
We can explore the content of this tree accessing to the attributes that start with the suffix ''to_''

In [23]:
mushrooms = big_tree.to_Fungi

Mushrooms is a TreeNode, i.e. not a complete TreeNeo object. To cast it (convert) to a TreeNode, and explore the families,classes, etc within it we must *plant-it* .

In [24]:
m = mushrooms.plantTreeNode()

In [25]:
type(mushrooms)

drivers.tree_builder.LocalTree

In [26]:
type(m)

drivers.tree_builder.TreeNeo

In [27]:
m.classes

[<TreeNode | Class: Agaricomycetes - n.count : 1- | AF: N.A. >,
 <TreeNode | Class: Pucciniomycetes - n.count : 3- | AF: N.A. >,
 <TreeNode | Class: Eurotiomycetes - n.count : 8- | AF: N.A. >,
 <TreeNode | Class: Lecanoromycetes - n.count : 91- | AF: N.A. >,
 <TreeNode | Class: Lichinomycetes - n.count : 6- | AF: N.A. >]

In [28]:
big_tree.to_Protozoa

<TreeNode | Kingdom: Protozoa - n.count : 17- | AF: N.A. >

In [29]:
big_tree.to_Plantae

<TreeNode | Kingdom: Plantae - n.count : 6441- | AF: N.A. >

In [30]:
big_tree.to_Animalia

<TreeNode | Kingdom: Animalia - n.count : 16291- | AF: N.A. >

### Cnidaria in the "terrestrial" environment ?  

In [32]:
cnidaria = big_tree.to_Animalia.to_Cnidaria

AttributeError: 'LocalTree' object has no attribute 'to_Cnidaria'

In [68]:
cnidaria

<TreeNode | Phylum: Cnidaria - n.count : 5- >

Where is this ?

In [69]:
cnidaria_cells = cnidaria.getExactCells()

In [70]:
cnidaria_cells

[<Cell id=223271>, <Cell id=199456>]

In [71]:
c1 , c2 = cnidaria_cells

In [72]:
c1.cell

u'MULTIPOLYGON (((-110.2690634159987 24.18688980100021, -110.2200634159987 24.18688980100021, -110.2200634159987 24.13788980100021, -110.2690634159987 24.13788980100021, -110.2690634159987 24.18688980100021)))'

![Cnidaria cell 1](cnidaria_cell1.png)

In [66]:
# c2.cell
## Dont do it, name to big.

![Cnidaria cell 2](cnidaria_cell2.png)

Close to the sea, but a little bit off the shore.

In [34]:
import redis

In [75]:
beatle = big_tree.to_Animalia.to_Arthropoda.to_Insecta.to_Coleoptera

## How many of these have beatles ?

In [35]:
with_beatles = filter(lambda tree : tree.hasNode(beatle),trees)

NameError: global name 'beatle' is not defined

In [36]:
len(with_beatles)

NameError: name 'with_beatles' is not defined

# Environmental Exploration
Remember that the big tree is defined in the geographical space given by the selected cells, i.e. the random sample.
We can get the environmental summary statistics of these cells with the following command.

In [37]:
%time bursera_env = big_tree.associatedData.getEnvironmentalVariablesCells(with_std=True)

CPU times: user 1min 42s, sys: 8.61 s, total: 1min 51s
Wall time: 3min 51s


In [38]:
bursera_env

{'MaxTemperature_mean': 14.121969454234756,
 'MaxTemperature_std': 4.006065262317484,
 'MeanTemperature_mean': 21.66893623838514,
 'MeanTemperature_std': 3.3072259058609785,
 'MinTemperature_mean': 29.20514258250562,
 'MinTemperature_std': 3.0664157452026277,
 'Precipitation_mean': 58.16749439282285,
 'Precipitation_std': 38.96332490543323,
 'SolarRadiation_mean': 18335.95265940404,
 'SolarRadiation_std': 1055.2620735583166,
 'Vapor_mean': 1.6413542667948304,
 'Vapor_std': 0.5536352334430281,
 'WindSpeed_mean': 2.3144558368044437,
 'WindSpeed_std': 0.5616478792620038}

### The same for point data

In [39]:
only_bursera = big_tree.to_Plantae.to_Magnoliophyta.to_Magnoliopsida.to_Sapindales.to_Burseraceae

In [40]:
pnts_env_burs = only_bursera.associatedData.getEnvironmentalVariablesPoints()

In [41]:
pnts_env_burs

,MaxTemperature_mean,MaxTemperature_std,MeanTemperature_mean,MeanTemperature_std,MinTemperature_mean,MinTemperature_std,Precipitation_mean,Precipitation_std,Vapor_mean,Vapor_std,SolarRadiation_mean,SolarRadiation_std,WindSpeed_mean,WindSpeed_std,x,y
0,14.500000,4.716991,23.250000,4.303584,31.750000,4.105383,17.583333,22.651190,1.833333,0.687184,18928.166667,3372.099243,3.000000,0.000000,-110.311,24.008
1,14.500000,4.716991,23.250000,4.303584,31.750000,4.105383,17.583333,22.651190,1.833333,0.687184,18928.166667,3372.099243,3.000000,0.000000,-110.311,24.008
2,18.500000,5.267827,25.166667,5.304610,31.666667,5.497474,18.333333,19.787763,1.750000,0.829156,18665.000000,4255.209317,3.000000,0.000000,-111.047,27.964
3,13.666667,6.073622,22.916667,6.075337,32.583333,5.823205,25.000000,27.658633,1.500000,0.645497,18472.000000,4512.020242,2.000000,0.000000,-110.956,29.001
4,13.666667,6.073622,22.916667,6.075337,32.583333,5.823205,25.000000,27.658633,1.500000,0.645497,18472.000000,4512.020242,2.000000,0.000000,-110.956,29.001
5,11.583333,5.073761,19.166667,4.758034,26.666667,4.515406,11.583333,7.576920,1.416667,0.493007,19196.250000,4439.565766,2.583333,0.493007,-113.910,29.160
6,15.166667,4.963758,22.750000,4.566636,30.416667,4.271189,19.583333,24.783927,1.750000,0.721688,18937.416667,3566.501892,3.000000,0.000000,-110.666,24.986
7,9.583333,2.722080,18.000000,2.645751,26.083333,2.752524,45.750000,42.575668,1.000000,0.000000,18881.833333,2361.063632,2.166667,0.372678,-100.267,20.600
8,15.333333,5.405758,21.000000,5.446712,26.583333,5.663602,11.666667,11.374924,1.750000,0.829156,18753.500000,4365.401232,3.000000,0.000000,-111.931,28.738
9,15.333333,5.405758,21.000000,5.446712,26.583333,5.663602,11.666667,11.374924,1.750000,0.829156,18753.500000,4365.401232,3.000000,0.000000,-111.931,28.738


## General aproach

In [72]:
r = redis.StrictRedis(host='redis', port=6379, db=0)

In [73]:
r.set('example2',pnts_env_burs.to_msgpack())

True

###  TO export into R
An option could be to use r2py but I've been experimeting with the Redis database. The idea is to store the dataframe as a CSV string and store it in a key : value .

In [79]:
r.set('example4',pnts_env_burs.to_csv())

True

In R it is very easy to read the data using the redisCMD package.
* string_dat = redisGet('example4')
con <- textConnection(string_dat)
data <- read.csv(con)

In [45]:
birds = t.to_Animalia.to_Chordata.to_Aves

In [46]:
bursera

<TreeNode type: Family id = 343770 name: Burseraceae>

In [47]:
birds

<TreeNode | Class: Aves - n.count : 8- >

In [48]:
trees

[<LocalTree Of Life | Root: LUCA - n.count : 14- >,
 <LocalTree Of Life | Root: LUCA - n.count : 4- >,
 <LocalTree Of Life | Root: LUCA - n.count : 37- >,
 <LocalTree Of Life | Root: LUCA - n.count : 373- >,
 <LocalTree Of Life | Root: LUCA - n.count : 5- >,
 <LocalTree Of Life | Root: LUCA - n.count : 9- >,
 <LocalTree Of Life | Root: LUCA - n.count : 2- >,
 <LocalTree Of Life | Root: LUCA - n.count : 59- >,
 <LocalTree Of Life | Root: LUCA - n.count : 947- >,
 <LocalTree Of Life | Root: LUCA - n.count : 1052- >]

In [49]:
filter(lambda tree : tree.hasNode(birds),trees)

[<LocalTree Of Life | Root: LUCA - n.count : 14- >,
 <LocalTree Of Life | Root: LUCA - n.count : 373- >,
 <LocalTree Of Life | Root: LUCA - n.count : 947- >,
 <LocalTree Of Life | Root: LUCA - n.count : 1052- >]

In [56]:
getNeighbourhood = lambda tree : tree.getNeighboringTrees()